In [1]:
from collections import defaultdict

In [2]:
data = open("input/16").read()

In [3]:
first, second = data.split("\n\n\n\n")

In [4]:
test_cases = []
for t in first.split("\n\n"):
    before, program, after = t.split("\n")
    before = eval(before.split("Before: ")[1])
    after = eval(after.split("After: ")[1])
    program = list(map(int, program.split(" ")))
    test_cases.append([before, after, program])

In [5]:
total_program = []
for line in second.split("\n"):
    program = list(map(int, line.split(" ")))
    total_program.append(program)

In [6]:
class Comp:
    def __init__(self, regs):
        self.regs = regs

        self.op_map = {
            0: self.addr,
            1: self.addi,
            2: self.mulr,
            3: self.muli,
            4: self.banr,
            5: self.bani,
            6: self.borr,
            7: self.bori,
            8: self.setr,
            9: self.seti,
            10: self.gtir,
            11: self.gtri,
            12: self.gtrr,
            13: self.eqir,
            14: self.eqri,
            15: self.eqrr,
        }

    def addr(self, a, b, c):
        self.regs[c] = self.regs[a] + self.regs[b]

    def addi(self, a, b, c):
        self.regs[c] = self.regs[a] + b

    def mulr(self, a, b, c):
        self.regs[c] = self.regs[a] * self.regs[b]

    def muli(self, a, b, c):
        self.regs[c] = self.regs[a] * b

    def banr(self, a, b, c):
        self.regs[c] = self.regs[a] & self.regs[b]

    def bani(self, a, b, c):
        self.regs[c] = self.regs[a] & b

    def borr(self, a, b, c):
        self.regs[c] = self.regs[a] | self.regs[b]

    def bori(self, a, b, c):
        self.regs[c] = self.regs[a] | b

    def setr(self, a, b, c):
        self.regs[c] = self.regs[a]

    def seti(self, a, b, c):
        self.regs[c] = a

    def gtir(self, a, b, c):
        self.regs[c] = 1 if a > self.regs[b] else 0

    def gtri(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] > b else 0

    def gtrr(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] >self.regs[b] else 0

    def eqir(self, a, b, c):
        self.regs[c] = 1 if a == self.regs[b] else 0

    def eqri(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] == b else 0

    def eqrr(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] == self.regs[b] else 0

In [7]:
def try_one(regs_before, regs_after, op, a, b, c):
    comp = Comp(regs_before)
    comp.op_map[op](a, b, c)
    return comp.regs == regs_after, comp.op_map[op].__name__

In [8]:
def check_one(regs_before, regs_after, program):
    op, a, b, c = program
    num_valid = 0
    op_codes = []
    for test_op in range(16):
        result, op_name = try_one(regs_before[:], regs_after, test_op, a, b, c)
        if result:
            op_codes.append(op_name)
            num_valid += 1
           
    return num_valid >= 3, op_codes

In [9]:
part1 = 0
total_valid = {c: [] for c in range(16)}
for before, after, program in test_cases:
    valid, op_codes = check_one(before, after, program)
    total_valid[program[0]].extend(op_codes)
    if valid:
        part1 += 1

In [10]:
assert part1 == 605
print(f"Answer #1: {part1}")

Answer #1: 605


In [11]:
operation_map = {}
for key, values in total_valid.items():
    operation_map[key] = list(set(values))

In [12]:
def prune_dict(operation_map):
    remove = []
    for val in operation_map.values():
        if len(val) == 1:
            remove.append(val[0])
    new = {}
    for key, val in operation_map.items():
        if len(val) == 1:
            new[key] = val
            continue
        subset = []
        for k in val:
            if k not in remove:
                subset.append(k)
        new[key] = subset
    return new

In [13]:
while True:
    operation_map = prune_dict(operation_map)
    for val in operation_map.values():
        if len(val) > 1:
            break
    else:
        break

In [14]:
comp = Comp([0, 0, 0, 0])
for code, a, b, c in total_program:
    getattr(comp, operation_map[code][0])(a, b, c)

In [15]:
part2 = comp.regs[0]

assert part2 == 653
print(f"Answer #2: {part2}")

Answer #2: 653
